# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

Откроем таблицу и посмотрим на ее содержимое.

In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/Anna/Desktop/Обучение/Аналитика Яндекс Практикум/Датасеты/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Описание данных:

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

In [2]:
data.info() #посмотрим на общую информацию о таблице.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В двух столбцах есть пропуски. Надо разобраться, почему они могли возникнуть и чем их можно заменить.

### Шаг 2.1 Заполнение пропусков

In [3]:
data.isna().sum() #находим, в каких столбцах есть пропущенные значения

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения обнаружились в двух столбцах: days_employed - стаж заёмщика в днях и total_income - доход. 
Одинаковое количество пропусков может означать, что значения отсутствуют по одним и тем же заёмщикам. Тогда это может говорить о том, что доход не указан именно потому, что человек не работает и никогда не работал. Но это гипотезу стоит проверить.

In [4]:
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)].info() 
#проверяем, совпадают ли пустые ячейки в двух ранее обозначенных столбцах

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Действительно, пустые ячейки в столбце с доходом совпадают с пустыми значениями в колонке со стажем.
Проверим, какую долю составляют пропущенные значения по отношению к общему числу значений.
Всего в таблице 21525 строк. Из них пустях строк в двух столбцах - 2174.

In [5]:
total_income_null = data['total_income'].isna().sum() / data['gender'].count()
total_income_null
'Доля пустых значений: {:.0%}'.format(total_income_null) #находим соотношение пустях строк ко всем стракам датафрейма.

'Доля пустых значений: 10%'

10% - слишком большая доля, чтобы просто избавиться от этих строк и проводить дальнейший анализ без них.
Значения в обоих столбцах с пропусками - количественные. Значит, целесообразно заменить пропуски на среднее значение. Но сначала надо проверить, какие минимальные и максимальные значения присутствуют в колонках с пропусками. Сначала разберемся с колонкой 'days_employed' - трудовой стаж в днях.

In [6]:
print(data['days_employed'].min() / 365) #минимальное значение стажа в годах
print(data['days_employed'].max() / 365) #максимальное значение стажа в годах
print(data['days_employed'].mean() / 365) #среднее арифметическое 
print(data['days_employed'].median() / 365) #медиана

-50.38068465909146
1100.6997273296713
172.73013057937976
-3.296902818549285


Значения очень странные: есть отрицательные значения, максимальное - гораздо больше, чем возможно в реальности. Среднее арифметическое - тоже невозможно. Медиана отрицательная. В реальной практике пришлось бы обратиться к разработчикам и выяснить, откуда в таблице такие данные по стажу. Возможно, выгрузка сделана с ошибкой, либо данные вносились неверно.
Такие данные можно считать ошибочными и не обрабатывать их.

Теперь посмотрим на данные в столбце total_income.

In [7]:
print(data['total_income'].min()) #минимальное значение дохода
print(data['total_income'].max()) #максимальное значение дохода
print(data['total_income'].mean()) #среднее арифметическое 
print(data['total_income'].median()) #медиана

20667.26379327158
2265604.028722744
167422.3022081719
145017.93753253992


Медиана среднемесячного дохода выглядит более реальной, чем среднее арифметическое, но не стоит забывать, что заёмщики имеют разный тип занятости. Возможно, лучше заменить пропущенные значения в столбце с доходом медианным значением по каждому типу отдельно.
Посчитаем, сколько уникальных типов занятости у нас в таблице.

In [8]:
print(data['income_type'].unique()) #выведем уникальные значения
data['income_type'].nunique() #посчитаем количество значений

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


8

In [9]:
data_grouped = data.groupby(['income_type']).agg({'total_income':'median'}) #выведем медиану по группам занятости
data_grouped

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


In [10]:
data['total_income'] = data.groupby(['income_type',], sort=False)['total_income'].apply(lambda x: x.fillna(x.median()))
#Заменим пропуски в столбце с доходом медианным значением, группируя по типу занятости.

In [11]:
data.info() #проверим, заменились ли пропуски в столбце total_income

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



Пропуски в столбце с доходом были заменены на медианные значения по типу занятости заёмщиков, так статистическая погрешность сводится к минимуму.

Пропуски в столбце days_employed остались без изменений, т.к. значения явно ошибочные, а в дальнейшем анализе не пригодятся.

Тот факт, что пропуски в двух столбцах находятся в одних и тех же строках, может говорить о том, что эти заёмщики не имеют официального дохода, т.к. не трудоустроены официально.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Мы уже выяснили, что в столбце days_employed присутствуют аномальные значения (отрицательные, а также неправдоподобно большие, сильно превышающие не только максимально возможный стаж, но и продолжительность жизни).

Проверим, есть ли аномальные значения в других столбцах.
Начнём со столбца children - количество детей.


In [12]:
print(data['children'].unique())
data['children'].nunique()

[ 1  0  3  2 -1  4 20  5]


8

В столбце с количеством детей обнаружились две аномалии: отрицательное значение и слишком большое значение.
Проверим, сколько таких значений в таблице.

In [13]:
sum(data['children'] == -1)

47

In [14]:
sum(data['children'] == 20)

76

Итого в таблице 47 сток со значением детей -1 и 76 строк со значением 20.
Количество небольшое, поэтому будем считать такие значения ошибочными и заменим -1 на 1, а 20 на 2.

In [15]:
data.loc[data['children'] == -1, 'children'] = 1 #заменим значение -1 на 1

In [16]:
sum(data['children'] == -1) #проверим, что значения -1 больше нет в таблице

0

In [17]:
data.loc[data['children'] == 20, 'children'] = 2 #заменим значение 20 на 2

In [18]:
sum(data['children'] == 20) #проверим, что значения 20 больше нет в таблице

0

С количеством детей разобрались, переходим к следующему столбцу.
Проверим данные в столбце dob_years - возраст клиента в годах.

In [19]:
print(data['dob_years'].min()) #минимальное значение возраста
print(data['dob_years'].max()) #максимальное значение возраста

0
75


Максимальный возраст выглядит нормально, а вот минимальный - 0, это аномалия. Проверим, сколько в столбце нулевых значений.

In [20]:
sum(data['dob_years'] == 0)

101

А теперь посмотрим, сколько значений больше 0, но меньше 18.

In [21]:
sum((data['dob_years'] > 0) & (data['dob_years'] < 18))

0

Таких нет. Значит, нужно заменить возраст на средний возраст заёмщика. Нулевых значений всего 101, диапазон возрастов ограничен (от 18 до 75 лет), заменим 0 на медианное значение по всей таблице, не разделя на категории.

In [22]:
data['dob_years'].median() #узнаем медианное значение по столбцу "возраст заёмщика"


42.0

In [23]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].median() #заменим нулевые значения на медианное

In [24]:
sum(data['dob_years'] == 0) #проверим, что нулевые значения исчезли.

0

Теперь проверим столбец "education" -  уровень образования клиента. Посмотрим, какие уникальные значения содержит этот столбец.

In [25]:
print(data['education'].unique())
data['education'].nunique()

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


15

В столбце с уровнем образования значения введены с использованием как верхнего, так и нижнего регистров.
Приведём все значения к нижнему регистру и проверим, сколько теперь уникальных значений.

In [26]:
data['education'] = data['education'].str.lower()


In [27]:
print(data['education'].unique())
data['education'].nunique()

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


5

Теперь всё правильно.
На всякий случай проверим столбец education_id.


In [28]:
print(data['education_id'].unique())
data['education_id'].nunique()

[0 1 2 3 4]


5

Тоже 5 значений, всё верно.

Проверим столбец family_status - семейное положение.

In [29]:
print(data['family_status'].unique())
data['family_status'].nunique()

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


5

В этом столбце значения корректные.

Проверим столбец gender - пол заёмщика.

In [30]:
print(data['gender'].unique())
data['gender'].nunique()

['F' 'M' 'XNA']


3

Есть строки со значением пола 'XNA'.
Посмотрим, сколько таких строк.

In [31]:
sum(data['gender'] == 'XNA')

1

Всего одно значение. По остальным колонкам в датафрейме невозможно определить пол. Удалим эту строку, т.к. на анализ одна строка сильно не повлияет.

In [32]:
data = data.loc[data['gender'] != 'XNA'] #удаляем из датафрейма строку с неопределенным полом
data['gender'].unique() #проверяем, что осталось только два значения в колонке 'gender'

array(['F', 'M'], dtype=object)

Осталось проверить колонку "purpose" - цель получения кредита

In [33]:
print(data['purpose'].unique())
data['purpose'].nunique()

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


38

В столбце 38 уникальных целей. Многие наименования - аналоги, можно "укрупнить" категории.
Возможно, заполнение этого поля лучше было бы организовать через выпадающий список из заранее заведенных категорий, а не через ручной ввод произвольных значений.

### Шаг 2.3. Изменение типов данных.

Для упрощения работы с многозначными числами заменим тип данных в столбце total_income на целочисленный.

In [34]:
data['total_income'] = data['total_income'].astype(int) #заменим тип данных на целочисленный
data['total_income'].dtypes #проверим, что тип изменился


dtype('int32')

### Шаг 2.4. Удаление дубликатов.

Проверим таблицу на наличие строк-дубликатов.

In [35]:
data.duplicated().sum() #посчитаем, сколько дубликатов в датафрейме

72

In [36]:
duplicated_data = data[data.duplicated()].head() #посмотрим на дубликаты
duplicated_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,NaN,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу


In [37]:
data = data.drop_duplicates().reset_index(drop=True) #удалим строки с дубликатами


In [38]:
print(data.duplicated().sum())
data.info() # проверим, что дубликаты удалены, а количество строк уменьшилось

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  int32  
 11  purpose           21452 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


Неявные дубликаты в столбце "education" - уровень образования клиента - были заменены ранее, в ходе поиска аномальных значений.
Такие дубликаты могли возникнуть из-за того, что поле "образование" заполняется вручную, а не выбирается из заранее заданных категорий - как и поле "цель кредита".

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим дополнительный датафрейм education_df, который будет содержать уровень образования клиента и идентификатор уровня образования.

In [39]:
education_df = data[['education_id','education']]
education_df.head(10) 

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


Уберём дубликаты.

In [40]:
education_df = education_df.drop_duplicates().reset_index(drop=True)
education_df.head() 

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Также создадим датафрейм из столбцов семейное положение и идентификатор семейного положения

In [41]:
family_df = data[['family_status_id','family_status']]
family_df.head(10) 

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


Уберем дубликаты.

In [42]:
family_df = family_df.drop_duplicates().reset_index(drop=True)
family_df.head() 

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Теперь из основного датафрейма можно удалить столбцы education и family_status, оставив их идентификаторы, 
к которым будем обращаться в дальнейшем.

In [43]:
data = data.drop('education', 1)


C:\Users\Anna\AppData\Local\Temp\ipykernel_9868\2261613360.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('education', 1)


In [44]:
data = data.drop('family_status', 1)

C:\Users\Anna\AppData\Local\Temp\ipykernel_9868\3523860935.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('family_status', 1)


In [45]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Создадим сводную таблицу, чтобы посмотреть, как распределяются случаи невозврата кредита в срок в зависимости от семейного положения. Частоту невозврата определим с помощью среднего арифметического по колонке "debt" - 0 означает возврат кредита в строк, 1 - случай невозврата.

In [46]:
data_pivot_status = data.pivot_table(index=['family_status_id'], values=["debt"], aggfunc=['sum', 'count', 'mean'])
data_pivot_status = data_pivot_status.sort_values(by=('mean', 'debt'))
data_pivot_status

,sum,count,mean
,debt,debt,debt
family_status_id,,,
2,63,959,0.065693
3,85,1195,0.071130
0,931,12339,0.075452
1,388,4149,0.093517
4,274,2810,0.097509


Самые ответственные заёмщики - вдовцы и разведенные. Больше всего должников среди заёмщиков в гражданском браке и не в браке.


### Шаг 2.6. Категоризация дохода.

Сгруппируем столбец с доходами заёмщиков по категориям:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

In [47]:
def income_category(total_income): #создадим функцию, в которой распишем доход по категорям
    
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <=1000000:
        return 'B'
    return 'A' 

In [48]:
income_category(25000) #проверим, что функция работает

'E'

In [49]:
data['total_income_category'] = data['total_income'].apply(income_category) #применим функцию, чтобы присвоить категории заёмщикам
data.head(10) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


In [50]:
data['total_income_category'].value_counts() #посмотрим распределение заёмщиков по уровню дохода

C    16014
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Получается, что реже всех берут кредиты клиенты из категорий с самым низким и самым высоким доходом.

Создадим сводную таблицу, чтобы посмотреть, как заёмщики возвращают кредиты в зависимости от уровня дохода.

In [51]:
data_pivot_income = data.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', 'mean'])
data_pivot_income = data_pivot_income.sort_values(by=('mean', 'debt'))
data_pivot_income

,sum,count,mean
,debt,debt,debt
total_income_category,,,
D,21,350,0.060000
B,356,5041,0.070621
A,2,25,0.080000
C,1360,16014,0.084926
E,2,22,0.090909


In [52]:
data_pivot_income.style.format({('mean', 'debt'): '{:.2%}'})

,sum,count,mean
,debt,debt,debt
total_income_category,,,
D,21,350,6.00%
B,356,5041,7.06%
A,2,25,8.00%
C,1360,16014,8.49%
E,2,22,9.09%


Меньше всего должников в группе с доходом чуть выше минимального. А больше всего - в группе с минимальным доходом. Явной зависимости между уровнем дохода и возвратом кредита в срок не наблюдается.

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая поможет нам сделать категории целей кредита более крупными, т.к. в соответствующем столбце содержится
много похожих названий.

In [53]:
def purpose_group(purpose):
    
    if ('жилья' in purpose or 'недвижимости' in purpose
        or 'недвижимостью' in purpose
        or 'недвижимость' in purpose
        or 'жилье' in purpose
        or 'жилью' in purpose
        or 'жильем' in purpose):
        return 'операции с недвижимостью'   
   
    elif ('автомобиля' in purpose or 'автомобилем' in purpose
        or 'автомобиль' in purpose
        or 'автомобили' in purpose):
        return 'операции с автомобилем'
    
    elif ('образование' in purpose or 'образования' in purpose
        or 'образованием' in purpose):
        return 'получение образования'
        
    elif ('свадьбу' in purpose or 'свадьбы' in purpose
        or 'свадьба' in purpose):
        return 'проведение свадьбы'
    

In [54]:
purpose_group('свадьба') #проверим, как работает функция

'проведение свадьбы'

Создадим новый столбец purpose_category с категориями целей
в зависимости от содержания в столбце с целью получения кредита. Это поможет укрупнить категории.

In [55]:
data['purpose_category'] = data['purpose'].apply(purpose_group)
data.head(10) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Теперь можно проверить, какое количество кредитов берут под конкретную категорию.

In [56]:
data['purpose_category'].value_counts()

операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

Наибольшее число кредитов приходится на операции с недвижимостью. Чем дороже стоит достижение цели, тем чаще люди обращаются за кредитом.

In [57]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', 'mean'])
data_pivot_purpose = data_pivot_purpose.sort_values(by=('mean', 'debt'))
data_pivot_purpose

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10810,0.072340
проведение свадьбы,186,2323,0.080069
получение образования,370,4013,0.092200
операции с автомобилем,403,4306,0.093590


Заёмщики, которые берут кредит для осуществления операций с недвижимостью, чаще остальных возвращают кредиты в срок. А больше всего должников среди тех, кто брал кредит под получение образования и операции с автомобилем.

Проанализируем, как часто клиенты возвращают кредит в срок в зависимости от количества детей.
Создадим сводную таблицу и проверим тем же методом, что и по другим категориям.

In [58]:
data_pivot_children = data.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', 'mean'])
data_pivot_children = data_pivot_children.sort_values(by=('mean', 'debt'))
data_pivot_children

,sum,count,mean
,debt,debt,debt
children,,,
5,0,9,0.000000
0,1063,14089,0.075449
3,27,330,0.081818
1,445,4855,0.091658
2,202,2128,0.094925
4,4,41,0.097561


Среди заемщиков, у которых 5 детей, должников нет, кредиты возвращены в срок. Но таких всего 9 человек, не слишком большая выборка, чтобы делать устойчивые выводы.
Среди тех, у кого нет детей, частота возвратов кредита в срок максимальная, а с увеличением количества детей частота падает, хотя среди заёмщиков с тремя детьми возврат кредита в срок происходит чаще, чем среди тех, у кого один ребенок или их двое.

### Ответы на вопросы.

##### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?
##### Вывод 1:
Прямой зависимости между количеством детей и возвратом кредита в срок нет, но те не менее, среди заёмщиков без детей средства возвращаются вовремя чаще.

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?
##### Вывод 2:
Семейное положение влияет на возврат кредита в срок. Самые ответственные заёмщики - вдовцы и разведенные. Больше всего должников среди заёмщиков в гражданском браке и не в браке.

##### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
##### Вывод 3:
Меньше всего должников в группе с доходом чуть выше минимального. А больше всего - в группе с минимальным доходом. 
Явной зависимости между уровнем дохода и возвратом кредита в срок не наблюдается, но наиболее рисковая категория - клиенты с низким уровнем дохода.

##### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?
##### Вывод 4:
Заёмщики, которые берут кредит для осуществления операций с недвижимостью, чаще остальных возвращают кредиты в срок. А больше всего должников среди тех, кто брал кредит под получение образования и операции с автомобилем.


## Общий вывод:

Проанализировав данные от банка, можно прийти к выводам:
1. Чаще всего клиенты обращаются за кредитом для проведения операций с недвижимостью.
2. Самые частые клиенты - люди в браке и без детей, их доход - средний. Это логично. Молодые семьи стараются обзавестись недвижимостью до рождения ребенка.

3. После проведения анализа по возврату кредита в срок в зависимости от нахождения в разных категориях можно нарисовать портрет идеального заёмщика:
- нет детей
- кредит нужен под операции с недвижимостью
- доход - выше среднего (группа B)
- статус - вдовцы и разведенные.

Неблагоприятный заёмщик:
- 4 детей
- кредит под операции с автомобилем
- минимальный доход (группа Е)
- не состоит в браке.